In [10]:
import os
import sys
sys.path.insert(0, "/home/ziyizhang/Desktop/Projects/mitsuba-curve/mitsuba3/build/python")

import mitsuba as mi
import drjit as dr

import numpy as np

mi.set_variant("cuda_ad_rgb")
# mi.set_variant("scalar_rgb")

In [17]:
T = mi.ScalarTransform4f
scene_dict = mi.cornell_box()
scene_dict["anisotropic"] = {
    'type': 'roughconductor',
    'material': 'Al',
    'distribution': 'ggx',
    'alpha_u': 0.05,
    'alpha_v': 0.35
}
scene_dict["curve1"] = {
    "type": "bspline",
    "filename": "data_samples/curve1.txt",
    "to_world": T.translate([0, 0.2, 0.2]) @ T.rotate([1, 0, 0], -40) @ T.scale(0.8),
    "bsdf": {
        "type": "ref",
        "id": "anisotropic"
    }
}
scene = mi.load_dict(scene_dict)

In [18]:
bsp = scene.shapes()[2]
params = mi.traverse(bsp)
params

SceneParameters[
  --------------------------------------------------------------------------
  Name                   Flags    Type            Parent
  --------------------------------------------------------------------------
  control_point_count             int             BSpline
  vertex                          Float           BSpline
  radius                          Float           BSpline
  bsdf.alpha_u.value     ∂, D     Float           UniformSpectrum
  bsdf.alpha_v.value     ∂, D     Float           UniformSpectrum
  bsdf.eta.value         ∂, D     Color3f         SRGBReflectanceSpectrum
  bsdf.k.value           ∂, D     Color3f         SRGBReflectanceSpectrum
]

In [19]:
image = mi.render(scene, spp=1024)
mi.Bitmap(image)

Bitmap[
  pixel_format = rgb,
  component_format = float32,
  size = [256, 256],
  srgb_gamma = 0,
  struct = Struct<12>[
    float32 R; // @0, premultiplied alpha
    float32 G; // @4, premultiplied alpha
    float32 B; // @8, premultiplied alpha
  ],
  data = [ 768 KiB of image data ]
]

## Curve generation

In [5]:
t = dr.linspace(mi.Float, 0, dr.pi * 12, 300)
t_max = dr.max(t)
x = dr.cos(t) * t / t_max
y = dr.sin(t) * t / t_max
z = t * 0
r = 0.03 + dr.sin(t / 3.0) * 0.02
pt = mi.Point3f(x, y, z)
np.savetxt("./data_samples/curve1.txt", np.array([x.numpy(), y.numpy(), z.numpy(), r.numpy()]).T)

In [6]:
import meshplot as mp
mp.plot(pt.numpy(), shading={"point_size": 0.1})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0418312…

## si

In [7]:
sensor = scene.sensors()[0]
viewpoint = sensor.world_transform() @ mi.ScalarPoint3f(0)
to_world = sensor.world_transform()
v_vert = to_world @ mi.Vector3f([0, 0, 1])
ray = mi.Ray3f(viewpoint + mi.Vector3f([0, 0.1, 0]), v_vert)

In [8]:
si = scene.ray_intersect(ray)
si.shape  # Assert that we hit a bsp curve

[BSpline[
  bsdf = SmoothDiffuse[
    reflectance = SRGBReflectanceSpectrum[
      value = [[0.885809, 0.698859, 0.666422]]
    ]
  ]
]]

In [9]:
print(si.p)  # its
print(si.cc)  # center of the disk (cross section)
print(dr.norm(si.p - si.cc))  # local radius

[[0.0, 0.10000000149011612, 0.3088795840740204]]
[[0.006528595462441444, 0.12272997200489044, 0.26483723521232605]]
[0.04999002441763878]
